In [1]:
import numpy as np
a = 10*(np.load('rateList300.npy')-np.load('rateList300.npy').min())#np.random.random(24) #  np.load('rateList300.npy')
a /= np.linalg.norm(a, ord=2)
Q=np.load('Q300.npy')#Q=np.random.random([n_features,n_features])+100*np.diag(np.ones(n_features))
hwithoutconstraints = lambda x:10*(x.dot(a))**1 #10*(x.dot(Q).dot(x))+

import argparse
from copy import deepcopy
import torch
import gym
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

import argparse
import sys

import copy
import numpy as np
from numpy.random import permutation
import matplotlib.pyplot as plt
import sys
sys.path.insert(0,'/root/gurobi903/linux64/lib/python3.6_utf32/gurobipy')

class para:
    def __init__(self):
        self.batch_size=16
        self.wmc=10
        self.solDim=300
        self.card=40
        self.test_batch_size=1000
        self.epochs=10
        self.lr=0.1
        self.momentum=0.5
        self.no_cuda='store_true'
        self.seed=1
        self.log_interval=10
        self.hidden_dim=10
args=para()
legalList=np.load('legalList300.npy',allow_pickle=True).item()
def checkFea(x):
    cnt=0
    cntT=0
    for i in range(len(legalList.keys())):
        for j in legalList[i]:
            cntT+=1
            if x[j]+x[i]>1:
                cnt+=1
    return cnt/cntT/2+abs(sum(x)-args.card)/args.card/2

In [2]:
from scipy.linalg import hadamard
import math

def feedback(x):
    rewardrec.append(hwithoutconstraints(x)) #(a.dot(x))**3
    constraintrec.append(checkFea(x))
    return hwithoutconstraints(x)-10*checkFea(x)
def reward(arms):
    res=np.zeros(len(N))
    for i in arms:
        res[i]=1
    return feedback(res)

In [3]:
def EST1(Nt,k,epsilont,deltat):
    n=len(Nt)
    m=2./epsilont/epsilont*np.log(2*n/deltat)
    if n%(2*k)==0:
        diversedSet=np.array(Nt).reshape(n//(2*k),2*k)
    elif n>2*k:
        diversedSet=[]
        for i in range(n//(2*k)):
            diversedSet.append(np.array(Nt[:2*k*(n//(2*k))]).reshape(n//(2*k),2*k)[i])
        diversedSet.append(list(Nt[-(n%(2*k)):])+list(np.random.choice(Nt[-(n%(2*k)):],size=2*k-n%(2*k))))
        diversedSet=np.array(diversedSet)
    else:
        diversedSet=[]
        diversedSet.append(list(Nt)+list(np.random.choice(Nt,size=2*k-n%(2*k))))
        diversedSet=np.array(diversedSet)
    H=hadamard(2**(int(math.log(2*k, 2))+1))[:(2*k)]
    H=np.array([i[:(2*k)] for i in H])
    S={}
    mu={}
    Z=np.zeros(2*k)
    theta={}

    for l in range(len(diversedSet)):
        currentN=list(set(diversedSet[l]))
        S[(0,-1)]=currentN[:k]
        S[(0,1)]=currentN[k:]
        for i in range(1,2*k):
            S[(i,1)]=list(set(np.argwhere(H[i]==1).reshape(len(np.argwhere(H[i]==1))))&set(currentN))
            S[(i,-1)]=list(set(np.argwhere(H[i]==-1).reshape(len(np.argwhere(H[i]==-1))))&set(currentN))
        for i in range(2*k):
            for b in [-1,1]:
                if len(S[(i,b)])==0:
                    mu[(i,b)]=0
                else:
                    np.random.shuffle(S[(i,b)])
                    mu[(i,b)]=np.array([reward(S[(i,b)][:k]) for _ in range(int(m))]).mean()
        Z[0]=mu[(0,1)]+mu[(0,-1)]
        for i in range(1,2*k):
            Z[i]=mu[(i,1)]-mu[(i,-1)]
        for index,arm in enumerate(currentN):
            theta[arm]=(H.T.dot(Z)/2/k)[index]
    return theta


for rnd in range(1):
    N=list(range(args.solDim))
    Nt=N.copy()
    Nt2=N.copy()
    At=[]
    At2=[]
    k=args.card
    epsilont=1/2
    delta=0.95
    delta1=4*delta/np.pi/np.pi
    deltat=delta1
    t=1
    rewardrec=[]
    constraintrec=[]
    while len(set(Nt)|set(At))>k and len(set(At))<k:    
        Nt=Nt2.copy()
        At=At2.copy() 
        thetat=EST1(Nt,k,max(epsilont,1e-8),deltat)
        A=[]
        R=[]
        if len(At)==k or len(rewardrec)>5000:
            break
        if len(Nt)>k:
            for i in Nt:
                if thetat[i]-np.sort(list(thetat.values()))[::-1][k]>2*epsilont:
                    A.append(i)
                if np.sort(list(thetat.values()))[::-1][k-1]-thetat[i]>2*epsilont:
                    R.append(i)
        else:   
            for i in Nt:
                if k-len(At)>=0 and thetat[i]-np.sort(list(thetat.values()))[::-1][k-len(At)]>2*epsilont:
                    A.append(i)
                if k-len(At)-1>=0 and np.sort(list(thetat.values()))[::-1][k-len(At)-1]-thetat[i]>2*epsilont:
                    R.append(i)
        At2=list(set(At)|set(A))
        Nt2=list(set(Nt)-(set(A)|set(R)))  
        print(len(At),len(Nt),len(A),len(R))
        epsilont=epsilont/2
        deltat=delta1/t/t
        t+=1
    np.save('reward_bandit1_1order300.npy',rewardrec)
    np.save('constraint_bandit1_1order300.npy',constraintrec)

In [6]:
def EST2(Nt,k,epsilont,deltat,A,T):
    n=len(Nt)
    k2=k-len(A)   
    m=2*k/k2/epsilont/epsilont*np.log(2*n/deltat)
    theta=EST1(T,k,epsilont,deltat)

    if n%(2*k2)==0:
        diversedSet=np.array(Nt).reshape(n//(2*k2),2*k2)
    elif n>2*k2:
        diversedSet=[]
        for i in range(n//(2*k2)):
            diversedSet.append(np.array(Nt[:2*k2*(n//(2*k2))]).reshape(n//(2*k2),2*k2)[i])
        diversedSet.append(list(Nt[-(n%(2*k2)):])+list(np.random.choice(Nt[-(n%(2*k2)):],size=2*k2-n%(2*k2))))
        diversedSet=np.array(diversedSet)
    else:
        diversedSet=[]
        diversedSet.append(list(Nt)+list(np.random.choice(Nt,size=2*k2-n%(2*k2))))
        diversedSet=np.array(diversedSet)
    H=hadamard(2**(int(math.log(2*k2, 2))+1))[:(2*k2)]
    H=np.array([i[:(2*k2)] for i in H])
    S={}
    mu={}
    Z=np.zeros(2*k2)
    thetat={}

    for l in range(len(diversedSet)):
        currentN=list(set(diversedSet[l]))
        S[(0,-1)]=list(set(currentN[:k2])|set(A))
        S[(0,1)]=currentN[k2:]
        for i in range(1,2*k2):
            S[(i,1)]=list(set(np.argwhere(H[i]==1).reshape(len(np.argwhere(H[i]==1))))&set(currentN))
            S[(i,-1)]=list(set(np.argwhere(H[i]==-1).reshape(len(np.argwhere(H[i]==-1))))&set(currentN))
        for i in range(2*k2):
            for b in [-1,1]:
                if len(S[(i,b)])==0:
                    mu[(i,b)]=0
                else:
                    mu[(i,b)]=np.array([reward(S[(i,b)][:k]) for _ in range(int(m))]).mean()
        Z[0]=mu[(0,1)]+mu[(0,-1)]-2*sum([theta[arm] for arm in A])
        for i in range(1,2*k2):
            Z[i]=mu[(i,1)]-mu[(i,-1)]
        for index,arm in enumerate(currentN):
            thetat[arm]=(H.T.dot(Z)/2/k2)[index]
    return thetat

for rnd in range(1):
    N=list(range(args.solDim))
    Nt=N.copy()
    Nt2=N.copy()
    At=[]
    At2=[]
    A=[]
    T=[]
    rewardrec=[]
    constraintrec=[]
    k=args.card
    epsilont=1/2
    delta=0.95
    delta1=4*delta/np.pi/np.pi
    deltat=delta1
    t=1

    while len(set(Nt)|set(At))>k and len(set(At))<k:
        Nt=Nt2.copy()
        At=At2.copy() 
        if len(A)>=10 or len(rewardrec)>5000:
            break
        if t==1:
            T=np.array(list(range(2*k)))
        else:
            T=np.array(list(A)+[i[0] for i in sorted(thetat.items(),key=lambda item:item[1])[::-1][:(2*k-len(A))]])
        thetat=EST2(Nt,k,max(epsilont,1e-8),deltat,A,T)
        A=[]
        R=[]
        if len(Nt)>k:
            for i in Nt:
                if thetat[i]-np.sort(list(thetat.values()))[::-1][k]>2*epsilont:
                    A.append(i)
                if np.sort(list(thetat.values()))[::-1][k-1]-thetat[i]>2*epsilont:
                    R.append(i)
        else:
            for i in Nt:
                if len(set(Nt)|set(At))-len(At)-1>=0 and np.sort(list(thetat.values()))[::-1][len(set(Nt)|set(At))-len(At)-1]-thetat[i]>2*epsilont:
                    R.append(i)
        At2=list(set(At)|set(A))
        Nt2=list(set(Nt)-(set(A)|set(R)))  
        print(len(At),len(Nt),len(A),len(R))
        epsilont=epsilont/2
        deltat=delta1/t/t
        t+=1
    np.save('reward_bandit2_1order300.npy',rewardrec)
    np.save('constraint_bandit2_1order300.npy',constraintrec)

0 300 0 0
